# Biol 359A | Familierize linear algebra
### Spring 2025, Week 8
Objectives:
- 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import (FloatSlider, Dropdown, VBox, HBox, interactive, Layout, 
                       Button, HTML, Accordion, Tab, Output)
from IPython import display